In [131]:
import pandas as pd
import yfinance as yf
import warnings

from support import fama_french

from interpret import show
from interpret.data import Marginal
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree
from interpret.perf import RegressionPerf

from sklearn.linear_model import LinearRegression as SKLin

In [132]:
fama_french(from_web=True)

,Mkt_RF,SMB,HML,RMW,CMA,MOM,RF
Date,,,,,,,
1963-07-01,-0.0039,-0.0041,-0.0097,0.0068,-0.0118,0.0090,0.0027
1963-08-01,0.0507,-0.0080,0.0180,0.0036,-0.0035,0.0101,0.0025
1963-09-01,-0.0157,-0.0052,0.0013,-0.0071,0.0029,0.0019,0.0027
1963-10-01,0.0253,-0.0139,-0.0010,0.0280,-0.0201,0.0312,0.0029
1963-11-01,-0.0085,-0.0088,0.0175,-0.0051,0.0224,-0.0074,0.0027
...,...,...,...,...,...,...,...
2022-04-01,-0.0946,-0.0040,0.0619,0.0363,0.0592,0.0489,0.0001
2022-05-01,-0.0034,-0.0006,0.0841,0.0144,0.0398,0.0248,0.0003
2022-06-01,-0.0843,0.0130,-0.0597,0.0185,-0.0470,0.0079,0.0006


In [136]:
aapl = yf.download("AAPL", interval="1mo", progress=False)
aapl.index = pd.to_datetime(aapl.index)
aapl = aapl.dropna()
ff = fama_french(from_web=True)

def set_first(df, inplace=False):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        df.loc[:,"Date"] = [f"{y}-{m}" for y, m in zip(df.index.year.to_list(), df.index.month.to_list())]
        df = df.reset_index(drop=True)
        df.loc[:,"Date"] = pd.to_datetime(df.loc[:,"Date"])
        df = df.set_index("Date", drop=True)
        if not inplace:
            return df

ff = set_first(ff)
aapl = set_first(aapl)

aapl.loc[:,"Ret"] = aapl.loc[:,"Adj Close"].pct_change()

df = pd.merge(aapl.loc[:,"Ret"], ff, left_index=True, right_index=True)

df = df.dropna()

df.tail()

,Ret,Mkt_RF,SMB,HML,RMW,CMA,MOM,RF
Date,,,,,,,,
2022-04-01,-0.097131,-0.0946,-0.0040,0.0619,0.0363,0.0592,0.0489,0.0001
2022-05-01,-0.055883,-0.0034,-0.0006,0.0841,0.0144,0.0398,0.0248,0.0003
2022-06-01,-0.080080,-0.0843,0.0130,-0.0597,0.0185,-0.0470,0.0079,0.0006
2022-07-01,0.188634,0.0957,0.0187,-0.0410,0.0068,-0.0694,-0.0396,0.0008
2022-08-01,-0.032552,-0.0378,0.0151,0.0031,-0.0480,0.0131,0.0209,0.0019


In [140]:
df["Ret"] -= df["RF"]
data = df.copy()
data = data.last("5Y")
# data.to_csv("../Data/apple_factor_returns.csv")
X, y = data.drop(["Ret", "RF"], axis=1), data[["Ret"]]

In [141]:
marginal = Marginal().explain_data(X, y, name = 'Data')

ebm = ExplainableBoostingRegressor(random_state=1, n_jobs=-1)
ebm.fit(X, y)

ebm_global = ebm.explain_global(name='EBM')

ebm_local = ebm.explain_local(X[:-5], y[:-5], name='EBM')

ebm_perf = RegressionPerf(ebm.predict).explain_perf(X, y, name='EBM')

lr = LinearRegression(linear_class=SKLin, fit_intercept=False)
lr.fit(X, y)

lr_a = LinearRegression(linear_class=SKLin, fit_intercept=True)
lr_a.fit(X, y)

rt = RegressionTree(random_state=1)
rt.fit(X, y)

lr_perf = RegressionPerf(lr.predict).explain_perf(X, y, name='Linear Regression No Intercept')
lr_a_perf = RegressionPerf(lr_a.predict).explain_perf(X, y, name='Linear Regression Intercept')
rt_perf = RegressionPerf(rt.predict).explain_perf(X, y, name='Regression Tree')

lr_global = lr.explain_global(name='Linear Regression No Intercept')
lr_a_global = lr_a.explain_global(name='Linear Regression Intercept')
rt_global = rt.explain_global(name='Regression Tree')

show([marginal, lr_global, lr_a_global, lr_perf, lr_a_perf, rt_global, rt_perf, ebm_global, ebm_perf])

<!-- http://127.0.0.1:7001/11269380352/ -->
 Open in new window

In [108]:
marginal = Marginal().explain_data(X, y, name = 'Data')
# show(marginal)

In [109]:
ebm = ExplainableBoostingRegressor(random_state=1, n_jobs=-1)
ebm.fit(X, y)

ExplainableBoostingRegressor(n_jobs=-1, random_state=1)

In [110]:
ebm_global = ebm.explain_global(name='EBM')
# show(ebm_global)

In [111]:
ebm_local = ebm.explain_local(X[:-5], y[:-5], name='EBM')
# show(ebm_local)

In [112]:
ebm_perf = RegressionPerf(ebm.predict).explain_perf(X, y, name='EBM')
# show(ebm_perf)

In [129]:
# lr = LinearRegression(random_state=1, alpha=0)
lr = LinearRegression(linear_class=SKLin)
lr.fit(X, y)

rt = RegressionTree(random_state=1)
rt.fit(X, y)

/Users/wyattnechtman/miniforge3/lib/python3.9/site-packages/interpret/glassbox/linear.py:66: UserWarning:

With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator

/Users/wyattnechtman/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning:

Coordinate descent with no regularization may lead to unexpected results and is discouraged.

/Users/wyattnechtman/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.990e-02, tolerance: 4.634e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.



In [126]:
lr_perf = RegressionPerf(lr.predict).explain_perf(X, y, name='Linear Regression')
rt_perf = RegressionPerf(rt.predict).explain_perf(X, y, name='Regression Tree')

show(lr_perf)
# show(rt_perf)
# show(ebm_perf)

<!-- http://127.0.0.1:7001/11253158048/ -->

In [127]:
lr_global = lr.explain_global(name='Linear Regression')
rt_global = rt.explain_global(name='Regression Tree')

show(lr_global)
# show(rt_global)
# show(ebm_global)

<!-- http://127.0.0.1:7001/11253789408/ -->

In [116]:
show([marginal, lr_global, lr_perf, rt_global, rt_perf, ebm_global, ebm_perf])

<!-- http://127.0.0.1:7001/11249740608/ -->
 Open in new window